In [22]:

import sys
import pandas as pd 
import tensorflow as tf 
# featuretools for automated feature engineering
import featuretools as ft
import featuretools.variable_types as vtypes

In [23]:
#df_train = pd.read_csv('D:/projects/ai/poc/homework/training_data_20210128.csv')
df_train = pd.read_csv('D:/projects/ai/poc/homework/testing_data_20210128.csv')

In [24]:
df_train.head()

# df_train = df_train.drop(columns=['IDX','LOT_TYPE','WS_DATE','ACTUAL_WP_OUT','DATA_DATE','OP_NO','LOT_ID'])
df_train = df_train.drop(df_train.loc[:, '0E':'ZL'].columns, axis = 1) 

In [25]:
es = ft.EntitySet(id = 'WIP')

In [26]:
# Entity set with id applications
es = ft.EntitySet(id = 'WIP')

# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'WIP',
             dataframe = df_train,  
             variable_types={'IDX': ft.variable_types.Categorical},
             make_index = True, 
             index = 'ID', time_index = 'DATA_DATE')

In [27]:
es

Entityset: WIP
  Entities:
    WIP [Rows: 1015, Columns: 16]
  Relationships:
    No relationships

In [28]:
# List the primitives in a dataframe
# primitives = ft.list_primitives()
# pd.options.display.max_colwidth = 100

# primitives[primitives['type'] == 'aggregation'].head(10)
primitives = ft.list_primitives()
#pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(22)

,name,type,dask_compatible,koalas_compatible,description
0,entropy,aggregation,False,False,Calculates the entropy for a categorical variable
1,all,aggregation,True,False,Calculates if all values are 'True' in a list.
2,max,aggregation,True,True,"Calculates the highest value, ignoring `NaN` values."
3,time_since_first,aggregation,False,False,Calculates the time elapsed since the first datetime (in seconds).
4,median,aggregation,False,False,Determines the middlemost number in a list of values.
5,trend,aggregation,False,False,Calculates the trend of a variable over time.
6,mode,aggregation,False,False,Determines the most commonly repeated value.
7,min,aggregation,True,True,"Calculates the smallest value, ignoring `NaN` values."
8,percent_true,aggregation,True,False,Determines the percent of `True` values.
9,first,aggregation,False,False,Determines the first value in a list.


In [29]:
primitives[primitives['type'] == 'transform'].head(100)

,name,type,dask_compatible,koalas_compatible,description
22,and,transform,True,True,Element-wise logical AND of two lists.
23,time_since_previous,transform,False,False,Compute the time since the previous entry in a list.
24,less_than,transform,True,True,Determines if values in one list are less than another list.
25,negate,transform,True,True,Negates a numeric value.
26,cum_max,transform,False,False,Calculates the cumulative maximum.
27,not_equal_scalar,transform,True,True,Determines if values in a list are not equal to a given scalar.
28,is_null,transform,True,True,Determines if a value is null.
29,isin,transform,True,True,Determines whether a value is present in a provided list.
30,cum_sum,transform,False,False,Calculates the cumulative sum.
31,less_than_equal_to,transform,True,True,Determines if values in one list are less than or equal to another list.


In [30]:
# Default primitives from featuretools
#default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
# default_trans_primitives =  ["day", "year", "month", "weekday", "haversine",   "characters"]
default_trans_primitives =  ["day", "year", "month", "weekday"]

In [31]:
# DFS with specified primitives
feature_names = ft.dfs(entityset = es, target_entity = 'WIP',
                       trans_primitives = default_trans_primitives,
                       #agg_primitives=default_agg_primitives, 
                        verbose = 1,
                       features_only = True)
feature_matrix,feature_names = ft.dfs(entityset = es, target_entity = 'WIP',
                       trans_primitives = default_trans_primitives,
                       #agg_primitives=default_agg_primitives, 
                        verbose = 1,
                       features_only = False)

 

Built 24 features
Built 24 features
Elapsed: 00:00 | Progress: 100%|██████████


In [32]:
feature_matrix,feature_names =ft.dfs(entityset = es, target_entity = 'WIP',
                       trans_primitives=['weekday', 'day'],
                       #agg_primitives=default_agg_primitives, 
                        verbose = 1,
                       features_only = False)


 

Built 18 features
Elapsed: 00:00 | Progress: 100%|██████████


In [33]:
#累績總計 與總數
 
feature_matrix, features_list = ft.dfs(entityset=es, 
    verbose = 1,
	target_entity='WIP',
    agg_primitives=[], 
	trans_primitives=['cum_sum', 'cum_count'], 
	groupby_trans_primitives=['cum_sum', 'cum_count'], #累積總計、累積總數
    
    ignore_variables={'WIP': ['PRIORITY']}
    
	)
feature_matrix    


Built 15 features
Elapsed: 00:00 | Progress: 100%|██████████


,LOT_ID,STATUS,CHIPNAME,LAYER,REMAIN_LAYER_SEQ,OP_NO,REMAIN_OP_SEQ,LOT_TYPE,WIP_QTY,IS_MAIN_ROUTE,IDX,CUM_SUM(IS_MAIN_ROUTE),CUM_SUM(REMAIN_LAYER_SEQ),CUM_SUM(REMAIN_OP_SEQ),CUM_SUM(WIP_QTY)
ID,,,,,,,,,,,,,,,
0,AWP462000,WAITING,CHIP25,PV,1.0,PV.WWH20,7.0,FDY,25,1,535017,1,1.0,7.0,25
1,AWP519000,WAITING,CHIP25,3M,4.0,3M.PPK10,71.0,FDY,25,1,535067,2,5.0,78.0,50
2,AWP520000,WAITING,CHIP25,5E,2.0,5E.EEH10,28.0,FDY,25,1,535068,3,7.0,106.0,75
3,AWP521000,WAITING,CHIP25,PO,3.0,PO.EEH10,42.0,FDY,25,1,535069,4,10.0,148.0,100
4,AWP522000,WAITING,CHIP25,PO,3.0,PO.EEH10,42.0,FDY,25,1,535070,5,13.0,190.0,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,AWP159000,WAITING,CHIP318,1T,11.0,1T.PPH10,282.0,FDY,25,1,534778,990,6440.0,152536.0,23786
1011,AWP426000,WAITING,CHIP318,1T,11.0,1T.EEH10,278.0,FDY,25,1,534975,991,6451.0,152814.0,23811
1012,AWR380000,WAITING,CHIP318,4T,5.0,4T.CDG20,111.0,FDY,25,1,536006,992,6456.0,152925.0,23836


In [34]:
ft.primitives.list_primitives()

,name,type,dask_compatible,koalas_compatible,description
0,entropy,aggregation,False,False,Calculates the entropy for a categorical variable
1,all,aggregation,True,False,Calculates if all values are 'True' in a list.
2,max,aggregation,True,True,"Calculates the highest value, ignoring `NaN` values."
3,time_since_first,aggregation,False,False,Calculates the time elapsed since the first datetime (in seconds).
4,median,aggregation,False,False,Determines the middlemost number in a list of values.
...,...,...,...,...,...
74,minute,transform,True,True,Determines the minutes value of a datetime.
75,or,transform,True,True,Element-wise logical OR of two lists.
76,multiply_numeric_scalar,transform,True,True,Multiply each element in the list by a scalar.
77,latitude,transform,False,False,Returns the first tuple value in a list of LatLong tuples.


In [35]:
from featuretools.primitives import CumCount
from featuretools.primitives import IsWeekend
#累績總數
cum_count = CumCount()
cum_count([1, 2, 3, 4, None, 5]).tolist()
isWkend = IsWeekend()
feature_matrix, features_list = ft.dfs(entityset=es, 
    verbose = 1,
	target_entity='WIP',
    agg_primitives=[], 
	trans_primitives=['cum_sum','cum_count','is_weekend'], 
	primitive_options={
        'cum_sum': {
                'include_variables':
                    {'WIP': ['WIP_QTY']},
                'ignore_variables':
                    {'WIP': ['IS_MAIN_ROUTE']}    
                    
                    },
		'cum_count': {
                'include_variables':
                    {'WIP': ['WIP_QTY']},
                'ignore_variables':
                    {'WIP': ['IS_MAIN_ROUTE']}    
                    
                    },
        'is_weekend': {
                'ignore_variables':
                        {'WIP': ['ACTUAL_WP_OUT']}}
        }
    
	)



feature_matrix    

Built 15 features
Elapsed: 00:00 | Progress: 100%|██████████


,LOT_ID,STATUS,CHIPNAME,LAYER,REMAIN_LAYER_SEQ,OP_NO,REMAIN_OP_SEQ,PRIORITY,LOT_TYPE,WIP_QTY,IS_MAIN_ROUTE,IDX,CUM_SUM(WIP_QTY),IS_WEEKEND(DATA_DATE),IS_WEEKEND(WS_DATE)
ID,,,,,,,,,,,,,,,
0,AWP462000,WAITING,CHIP25,PV,1.0,PV.WWH20,7.0,4,FDY,25,1,535017,25,True,False
1,AWP519000,WAITING,CHIP25,3M,4.0,3M.PPK10,71.0,4,FDY,25,1,535067,50,True,False
2,AWP520000,WAITING,CHIP25,5E,2.0,5E.EEH10,28.0,4,FDY,25,1,535068,75,True,False
3,AWP521000,WAITING,CHIP25,PO,3.0,PO.EEH10,42.0,4,FDY,25,1,535069,100,True,False
4,AWP522000,WAITING,CHIP25,PO,3.0,PO.EEH10,42.0,4,FDY,25,1,535070,125,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,AWP159000,WAITING,CHIP318,1T,11.0,1T.PPH10,282.0,4,FDY,25,1,534778,23786,True,True
1011,AWP426000,WAITING,CHIP318,1T,11.0,1T.EEH10,278.0,3,FDY,25,1,534975,23811,True,False
1012,AWR380000,WAITING,CHIP318,4T,5.0,4T.CDG20,111.0,1,FDY,25,1,536006,23836,True,False


In [36]:
#select features that have at least 2 unique values and that are not all null
#(feature_matrix, features=None):
from featuretools.selection import remove_low_information_features  ## 移除较小信息的特征

#Removes columns from a feature matrix that have higher than a set threshold of null values.
#(feature_matrix, features=None, pct_null_threshold=0.95):
from featuretools.selection import remove_highly_null_features

#Removes columns in feature matrix where all the values are the same.
#(feature_matrix, features=None, count_nan_as_value=False):
from featuretools.selection import remove_single_value_features

#Removes columns in feature matrix that are highly correlated with another column.
#(feature_matrix, features=None, pct_corr_threshold=0.95,features_to_check=None, features_to_keep=None):
from featuretools.selection import remove_highly_correlated_features 

print('features',feature_matrix.shape)
features1, feature_names1 = remove_low_information_features(feature_matrix, features_list)
print('remove_low_information_features',feature_matrix.shape)
print('Removed %d features from training features' % (feature_matrix.shape[1] - features1.shape[1])) 


features1, feature_names1 = remove_highly_null_features(feature_matrix, features_list,0.95)
print('remove_highly_null_features',features1.shape)
print('Removed %d features from training features' % (feature_matrix.shape[1] - features1.shape[1])) 

features1, feature_names1 = remove_single_value_features(feature_matrix, features_list)
print('remove_single_value_features',features1.shape)
print('Removed %d features from training features' % (feature_matrix.shape[1] - features1.shape[1])) 

features1, feature_names1 = remove_highly_correlated_features(feature_matrix, features_list,0.95)
print('remove_highly_correlated_features',features1.shape)
print('Removed %d features from training features' % (feature_matrix.shape[1] - features1.shape[1])) 

features (1015, 15)
remove_low_information_features (1015, 15)
Removed 1 features from training features
remove_highly_null_features (1015, 15)
Removed 0 features from training features
remove_single_value_features (1015, 14)
Removed 1 features from training features
remove_highly_correlated_features (1015, 14)
Removed 1 features from training features


In [37]:
#featuretools.synthesis :encode_features
fm_enc,f_enc =ft.encode_features(features1, feature_names1)
fm_enc.head()

,LOT_ID = AWT428000,LOT_ID = AWS731000,LOT_ID = AWS665000,LOT_ID = AWS468000,LOT_ID = AWS401000,LOT_ID = AWS327000,LOT_ID = AWS267000,LOT_ID = AWS128000,LOT_ID = AWS126000,LOT_ID = AWS125000,...,IDX = 538740,IDX = 538712,IDX = 538651,IDX = 538586,IDX = 538427,IDX = 538426,IDX is unknown,CUM_SUM(WIP_QTY),IS_WEEKEND(DATA_DATE),IS_WEEKEND(WS_DATE)
ID,,,,,,,,,,,,,,,,,,,,,
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,25,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,50,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,75,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,100,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,125,True,False


In [49]:
from featuretools.primitives import make_agg_primitive, make_trans_primitive
from featuretools.variable_types import Text, Numeric
# def absolute(column):	
#     return abs(column)

# Absolute = make_trans_primitive(function=absolute, input_types=[Numeric], return_type=Numeric)

# def maximum(column):	
#     return max(column)

# Maximum = make_agg_primitive(function=maximum, input_types=[Numeric], return_type=Numeric)

# feature_matrix, feature_defs = ft.dfs(	entityset=es,	
#                 target_entity="clients",	
#                 agg_primitives=[Maximum],	
#                 trans_primitives=[Absolute],	
#                 max_depth=2) 


SyntaxError: invalid syntax (<ipython-input-49-ca98541940d4>, line 2)